In [1]:
import os
import pandas as pd
import numpy as np

from vizro import Vizro; Vizro._reset()
import vizro.models as vm
from vizro.models.types import capture
import vizro.plotly.express as px
import plotly.graph_objects as go
import plotly.express as pxo

from google.cloud import bigquery

from scipy.optimize import curve_fit

In [3]:
client = bigquery.Client()

In [4]:
data_availability = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.data_availability`").to_dataframe()

In [5]:
diurnal_daily = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.diurnal_variation_daily`").to_dataframe()
diurnal_monthly = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.diurnal_variation_monthly`").to_dataframe()
diurnal_yearly = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.diurnal_variation_yearly`").to_dataframe()

In [6]:
rose_hour_daily = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.wr_daily_hour`").to_dataframe()
rose_hour_monthly = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.wr_monthly_hour`").to_dataframe()
rose_hour_yearly = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.wr_yearly_hour`").to_dataframe()

In [7]:
rose_raw_daily_df = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.wr_daily`").to_dataframe()
rose_raw_monthly_df= client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.wr_monthly`").to_dataframe()
rose_raw_yearly_df = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.wr_yearly`").to_dataframe()

In [8]:
distribution_monthly_df = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.frequency_distribution_monthly`").to_dataframe()
distribution_yearly_df = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.frequency_distribution_yearly`").to_dataframe()

In [9]:
wind_stats = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.monthly_stats`").to_dataframe()
stats_min = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.min_wind_instances`").to_dataframe()
stats_max = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.max_wind_instances`").to_dataframe()

In [10]:
wind_shear = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.delta_wind_height`").to_dataframe()
weibull = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.weibull`").to_dataframe()
yey = client.query("select * from `wired-ripsaw-403910.dbt_gbabeleda.yey`").to_dataframe()

In [ ]:
aug_k = weibull[weibull['months'] == 8]

In [ ]:
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

def fit_weibull(df : pd.DataFrame, k_initial : float, mse_limit : float, max_iterations : int = 50):
    
    def weibull_function_k(v : float, k : float, A : float):
        
        return (k / A) * (v / A) ** (k - 1) * np.exp(-(v / A) ** k)
    
    v = df['speed_at_turbine'].values
    vm = df['wind_shear'].iloc[0]
    f_v_exp = df['f_v'].values
    
    A = (2 / np.sqrt(np.pi)) * vm
    
    k = k_initial
    
    mse = mean_squared_error(f_v_exp,weibull_function_k(v=v,k=k_initial,A=A))
    print(f"Initial k : {k_initial}, Initial MSE : {mse}")
    
    for iteration in range(max_iterations):
        if mse <= mse_limit:
            break
            
        optimized_parameters, _ = curve_fit(weibull_function_k, v, f_v_exp, p0=[k])
        k = optimized_parameters[0]
        
        mse = mean_squared_error(f_v_exp,weibull_function_k(v,k,A))
        
        print(f"Iteration {iteration + 1}: k = {k}, MSE = {mse}")
        
    f_v_calc = weibull_function_k(v,k,A)
    
    return k, mse, f_v_exp,f_v_calc

In [ ]:
k_analysis = fit_weibull(df=aug_k,k_initial=2,mse_limit=0.001)

Initial k : 2, Initial MSE : 4.3177457545786855e-34


In [ ]:
k_viz_df = pd.DataFrame(
    {
        'v' : aug_k['speed_at_turbine'],
        'f(v) experimental' : k_analysis[2],
        'f(v) calculated' : k_analysis[3]
    }
)

# Dashboard

In [ ]:
Vizro._reset()

In [ ]:
landing_page = vm.Page(
    title="Wind Resource Assessment Dashboard",
    layout=vm.Layout(
        grid=[
            [0,0],
            [1,1],
            [2,3],

        ],
        row_min_height="500px"
    ),
    components=[
        vm.Card(
            id="Banner",
            text=
            """ 
            ![Banner](assets/dalle-wind-farm.png#banner)
            """
        ),
        vm.Card(
            id="Data Acknowledgement",
            text=
            """ 
            # Data Declaration and Acknowledgement
            The data used in this dashboard was provided via the Geodetic Engineering 
            Component of the Energy Engineering 205 Laboratory class presided over by 
            Engineer Rosario Ang, under the Energy 
            Engineering Program of the University of the Philippines Dilimam
            
            The author of the dashboard would like to:
            - Acknowledge the Applied Geodesy and Space Technology Research Laboratory (AGST Lab)
            of the UP Training Center for Applied Geodesy & Photogrammetry (TCAGP), College of 
            Engineering, UP Diliman. 
            - Express that this data is only used for research, and to showcase skill in wind
            resource assessment, and the use of PostgreSQL, Python, and related tools. 
           
           
            
            ![Data Declaration](assets/data_declaration.png#image)
            """
        ),
        vm.Card(
            id="Overview, Navigation",
            text=
            """ 
            # Welcome!
            
            This project aims to create a dashboard to aid in determining
            the theoretical wind energy potential of a site and to showcase
            proficiency in certain technical skills in creating a dashboard
            
            This dashboard can theoretically be applied to any wind site. 
            
            Skills and Tools
            - Python
            - SQL
            - Git and Github
            - PostgreSQL
            - Structuring a programming project (file structure, virtual environments, etc)
            - Jupyter Notebooks
            - Magic Commands
            - Pandas
            - Psycopg
            - Dashboarding
            - Vizro (Dashboarding Tool)
            """
        ),
        vm.Card(
            id="Author Information",
            text=
            """ 
            Author Name: Jose Mari Angelo Abeleda Jr \n
            Contact Number: 0968 681 1458 \n
            Email: gioabeleda@gmail.com \n
            Github: https://github.com/gbabeleda \n   
            """
        )
    ]
)

In [ ]:
methodology_page = vm.Page(
    title="Methodology",
    layout=vm.Layout(
        grid=[
            [0],
            [1]
        ],
        row_min_height="500px"
    ),
    components=[
        vm.Card(
            id="Methodology Figure",
            text=
            """ 
            Placeholder
            """
        ),
        vm.Card(
            id="Methodology Text",
            text=
            """ 
            # Methodology
            
            - A postgresql server, database, schema and table were setup
            - Pandas was used to remove rows with null values and to create a csv from an excel file containing the data
            - The csv was loaded into the postgres using !psql magic commands
            - Queries were done using '%SQL' magic commands
            - Vizro was used to build the dashboard and to generate visualizations
            """
        ) 
    ]
)

In [ ]:
data_availability_page = vm.Page(
    title="Data Availability",
    layout=vm.Layout(
        grid=[
            [0,1]
        ],
        row_min_height="500px",
    ),
    components=[
        vm.Graph(
            id="Data Availability Graph",
            figure=px.bar(
                data_frame=data_availability,
                y="year_month",
                x="count_days",
                color="count_days",
                labels={"year_month" : "Month", "count_days" : "Days Counted"},
                color_continuous_scale="sunset",
                text="count_days"
            )
        ),
        vm.Card(
            id="Data Availability Text",
            text=
            """ 
            # Data Availability
            
            We define a unique day having data available if even a single non-null
            wind record is present for that day
            
            This means that there are days in this data set that may not contain wind
            complete wind records per day. 

            As we can see, data for both January and March 2010 are incomplete,
            with there being no data at all for the month of February. Thus, we cannot
            do analysis using these months as they are not representative of the population.
            """
        ) 
    ]
)

In [ ]:
diurnal_page = vm.Page(
    title="Diurnal Variation",
    layout=vm.Layout(
        grid=[
            [0,0],
            [1,2]
        ],
        row_min_height="500px",
    ),
    components=[
        vm.Graph(
            id="Daily Diurnal",
            figure=px.scatter(
                data_frame=diurnal_daily,
                x="hours",
                y="avg_wind_speed",
                color="avg_wind_speed",
                labels={"avg_wind_speed" : "Mean Hourly Wind Speed", "hours" : "Hour"},
                title="Daily",
                
            )
        ),
        vm.Graph(
            id="Monthly Diurnal",
            figure=px.scatter(
                data_frame=diurnal_monthly,
                x="hours",
                y="avg_wind_speed",
                color="avg_wind_speed",
                labels={"avg_wind_speed" : "Mean Hourly Wind Speed", "days" : "Days", "hours" : "Hour"},
                title="Monthly"
                
            )
        ),
        vm.Graph(
            id="Yearly Diurnal",
            figure=px.scatter(
                data_frame=diurnal_yearly,
                x="hours",
                y="avg_wind_speed",
                color="avg_wind_speed",
                labels={"avg_wind_speed" : "Mean Hourly Wind Speed", "hours" : "Hour"},
                title="Yearly"
                
            )
        )       
    ],
    controls=[
        vm.Filter(
            column="years",
            targets=["Yearly Diurnal","Monthly Diurnal","Daily Diurnal"],
            selector=vm.RadioItems()
        ),
        vm.Filter(
            column="months",
            targets=["Monthly Diurnal","Daily Diurnal"],
            selector=vm.Slider(
                step=1
            )
        ),
        vm.Filter(
            column="days",
            targets=["Daily Diurnal"],
            selector=vm.RadioItems()
        )
    ]
)

In [ ]:
wind_rose_hourly_page = vm.Page(
    title="Wind Rose Hour",
    layout=vm.Layout(
        grid=[
            [0,1],
            [2,2]
        ],
        row_min_height="500px"   
    ),
    components=[
        vm.Graph(
            id="Wind Rose Day",
            figure=px.bar_polar(
                data_frame=rose_hour_daily,
                r="cumulative_perc_freq",
                theta="cardinal_direction",
                color="speed_bin",
                barmode="group",
                title="Daily",
                labels={"speed_bin" : "Wind Speed Bin"},
                category_orders={"cardinal_direction" : ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']},
                color_continuous_scale="blugrn"
                
            )
        ),
        vm.Graph(
            id="Wind Rose Month",
            figure=px.bar_polar(
                data_frame=rose_hour_monthly,
                r="cumulative_perc_freq",
                theta="cardinal_direction",
                color="speed_bin",
                barmode="group",
                title="Monthly",
                labels={"speed_bin" : "Wind Speed Bin"},
                category_orders={"cardinal_direction" : ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']},
                color_continuous_scale="emrld"
            )
        ),
        vm.Graph(
            id="Wind Rose Year",
            figure=px.bar_polar(
                data_frame=rose_hour_yearly,
                r="cumulative_perc_freq",
                theta="cardinal_direction",
                color="speed_bin",
                barmode="group",
                title="Yearly",
                labels={"speed_bin" : "Wind Speed Bin"},
                category_orders={"cardinal_direction" : ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']},
                color_continuous_scale="deep"
            )
        )
        
    ],
    controls=[
        vm.Filter(
            column="years",
            targets=[
                "Wind Rose Year",
                "Wind Rose Month",
                "Wind Rose Day"
                ],
            selector=vm.RadioItems()
        ),
        vm.Filter(
            column="months",
            targets=[
                "Wind Rose Month",
                "Wind Rose Day"
                ],
            selector=vm.Slider(
                step=1
            )
        ),
        vm.Filter(
            column="days",
            targets=[
                "Wind Rose Day"
                ],
            selector=vm.RadioItems()
        )
    ]
)

In [ ]:
wind_rose_raw_page = vm.Page(
    title="Wind Rose Raw",
    layout=vm.Layout(
        grid=[
            [0,1],
            [2,2]
        ],
        row_min_height="500px"   
    ),
    components=[
        vm.Graph(
            id="Wind Rose Day Raw",
            figure=px.bar_polar(
                data_frame=rose_raw_daily_df,
                r="cumulative_perc_freq",
                theta="cardinal_direction",
                color="speed_bin",
                barmode="group",
                title="Daily",
                labels={"speed_bin" : "Wind Speed Bin"},
                category_orders={"cardinal_direction" : ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']},
                color_continuous_scale="blugrn"            
            )
        ),
        vm.Graph(
            id="Wind Rose Month Raw",
            figure=px.bar_polar(
                data_frame=rose_raw_monthly_df,
                r="cumulative_perc_freq",
                theta="cardinal_direction",
                color="speed_bin",
                barmode="group",
                title="Monthly",
                labels={"speed_bin" : "Wind Speed Bin"},
                category_orders={"cardinal_direction" : ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']},
                color_continuous_scale="emrld"
            )
        ),
        vm.Graph(
            id="Wind Rose Year Raw",
            figure=px.bar_polar(
                data_frame=rose_raw_yearly_df,
                r="cumulative_perc_freq",
                theta="cardinal_direction",
                color="speed_bin",
                barmode="group",
                title="Yearly",
                labels={"speed_bin" : "Wind Speed Bin"},
                category_orders={"cardinal_direction" : ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']},
                color_continuous_scale="deep"
            )
        )
    ],
    controls=[
        vm.Filter(
            column="years",
            targets=[
                "Wind Rose Year Raw",
                "Wind Rose Month Raw",
                "Wind Rose Day Raw"
                ],
            selector=vm.RadioItems()
        ),
        vm.Filter(
            column="months",
            targets=[
                "Wind Rose Month Raw",
                "Wind Rose Day Raw"
                ],
            selector=vm.Slider(
                step=1
            )
        ),
        vm.Filter(
            column="days",
            targets=[
                "Wind Rose Day Raw"
                ],
            selector=vm.RadioItems()
        )
    ]
)

In [ ]:
freq_dist_page = vm.Page(
    title="Frequency Distribution",
    layout=vm.Layout(
        grid=[
            [0],
            [1]
        ],
        row_min_height="500px"   
    ),
    components=[
        vm.Graph(
            id="Monthly Distribution",
            figure=px.bar(
                data_frame=distribution_monthly_df,
                x="speed_bin",
                y="percent_frequency",
                color="percent_frequency",
                labels={"speed_bin" : "Wind Speed Bins", "percent_frequency" : "Frequency (%)"},
                hover_name="year_month",
                title="Monthly",
                color_continuous_scale="viridis",
                text="percent_frequency"
            )
        ),
        vm.Graph(
            id="Yearly Distribution",
            figure=px.bar(
                data_frame=distribution_yearly_df,
                x="speed_bin",
                y="percent_frequency",
                color="percent_frequency",
                labels={"speed_bin" : "Wind Speed Bins", "percent_frequency" : "Frequency (%)"},
                hover_name="years",
                title="Yearly",
                color_continuous_scale="viridis",
                text="percent_frequency"
            )
        )
    ],
    controls=[
        vm.Filter(
            column="years",
            targets=["Yearly Distribution","Monthly Distribution"],
            selector=vm.RadioItems()
        ),
        vm.Filter(
            column="months",
            targets=["Monthly Distribution"],
            selector=vm.Slider(
                step=1
            )
        )
    ]
)

In [ ]:
stats_page = vm.Page(
    title="Statistics",
    layout=vm.Layout(
        grid=[
            [0,1],
            [2,2],
            [3,4]
        ],
        row_min_height="500px" 
    ),
    components=[
        vm.Graph(
            id="Maximum Speed Monthly",
            figure=px.bar(
                data_frame=wind_stats,
                x="max_speed",
                y="year_month",
                color="max_speed",
                labels={"year_month" : "Month", "max_speed" : "Max Speed"},
                title="Maximum Wind Speeds Per Month",
                text="max_speed",
                color_continuous_scale="plotly3"
            )
        ),
        vm.Graph(
            id="Minimum Speed Monthly",
            figure=px.scatter(
                data_frame=wind_stats,
                x="min_speed",
                y="year_month",
                color="min_speed",
                labels={"year_month" : "Month", "min_speed" : "Min Speed"},
                title="Mininimum Wind Speeds Per Month",
                color_continuous_scale="plasma"
            )
        ),
        vm.Graph(
            id="Average Speed Monthly",
            figure=px.bar(
                data_frame=wind_stats,
                y="avg_speed",
                x="year_month",
                color="avg_speed",
                labels={"year_month" : "Month", "avg_speed" : "Avg Speed"},
                title="Average Wind Speeds Per Month",
                color_continuous_scale="dense",
                text="avg_speed",
            )
        ),
        vm.Graph(
            id="Stats Hourly Max",
            figure=px.scatter(
                data_frame=stats_max,
                x="year_month",
                y="hours",
                color="days",
                size="max_speed",
                hover_data=["year_month","max_speed"],
                labels={"hours": "Hour", "days": "Day", "msc_speed": "Wind Speed", "year_month" : "Month"},
                title="Occurences of Maximum Wind Speed",
                color_continuous_scale="twilight"
            )
        ),
        vm.Graph(
            id="Stats Hourly Min",
            figure=px.scatter(
                data_frame=stats_min,
                x="days",
                y="hours",
                color="hours",
                hover_data=["year_month","min_speed"],
                labels={"hours": "Hour", "days": "Day", "min_speed": "Wind Speed"},
                title="Occurences of Minimum Wind Speed",
                color_continuous_scale="plasma"
            )
        )      
    ],
    controls=[
        vm.Filter(
            column="year_month",
            targets=["Stats Hourly Min"],
            selector=vm.RadioItems()
        ),
    ]
)

In [ ]:
weibull_page = vm.Page(
    title="Weibull Distribution and Periodic Energy Production",
    layout=vm.Layout(
        grid=[
            [0,0],
            [1,2],
            [3,3],
            [4,4],
            [5,5]
        ],
        row_min_height="500px"
    ),
    components=[
        vm.Graph(
            id="Wind Shear",
            figure=px.bar(
                data_frame=wind_shear,
                x="year_month",
                y="wind_shear",
                barmode="group",
                labels={
                    "value" : "Wind Speed (m/s)",
                    "wind_shear" : "Wind Speed @ Rotor",
                    "year_month" : "Month"
                },
                title="Wind Shear Per Month",
            )
        ),
        vm.Card(
            id="Wind Turbine Details",
            text=
            """ 
            # Wind Turbine Details
            Manufacturer: Vestas \n
            Model: V20/100 \n
            Rated Power: 100 kW \n
            Rotor Diameter: 20 m \n
            Cut-in wind speed: 5 m/s \n
            Rated wind speed: 17.5 m/s \n
            Cut-off wind speed: 25 m/s \n
            Hub Height: 24 m \n
            """
        ),
        vm.Graph(
            id="Power Curve",
            figure=px.scatter(
                data_frame=weibull[weibull["months"] == 10],
                x="speed_at_turbine",
                y="power_curve",
                labels={
                    "power_curve" : "Power (kW)",
                    "speed_at_turbine" : "Wind speed (m/s)"
                },
                title="Vestas 20/100 Power Curve"
            )
        ),
        vm.Graph(
            id="Weibull Function",
            figure=px.scatter(
                data_frame=weibull,
                x="speed_at_turbine",
                y="f_v",
                labels={
                    "f_v" : "f(v)",
                    "speed_at_turbine" : "Wind speed (m/s)"
                },
                title="Weibull Function per Month"
            )
        ),
        vm.Graph(
            id="Periodic Energy Production Daily",
            figure=px.bar(
                data_frame=yey,
                x="year_month",
                y="sum_yey_daily",
                labels={
                    "year_month" : "Month",
                    "sum_yey_daily" : "Daily Energy Production (kWh)"
                },
                color="sum_yey_daily",
                text="sum_yey_daily"
            )
        ),
        vm.Graph(
            id="Periodic Energy Production Yearly",
            figure=px.bar(
                data_frame=yey,
                x="year_month",
                y="sum_yey_yearly",
                labels={
                    "year_month" : "Month",
                    "sum_yey_yearly" : "Yearly Energy Production (kWh)"
                },
                color="sum_yey_yearly",
                text="sum_yey_yearly"
            )
        ),
    ],
    controls=[
        vm.Filter(
            column="years",
            targets=["Wind Shear","Weibull Function","Periodic Energy Production Daily", "Periodic Energy Production Yearly"],
            selector=vm.RadioItems()
        ),
        
        vm.Filter(
            column="months",
            targets=["Wind Shear","Weibull Function"],
            selector=vm.Slider(
                step=1
            )   
        )
    ]
)

In [ ]:
dashboard = vm.Dashboard(
    pages=[
        landing_page,
        methodology_page,
        data_availability_page,
        diurnal_page,
        wind_rose_hourly_page,
        wind_rose_raw_page,
        freq_dist_page,
        stats_page,
        weibull_page,
    
    ]
)

In [ ]:
Vizro().build(dashboard=dashboard).run(port="8053")